In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
sys.path.append(grandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.MLP import MLP
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_MLP = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D_list).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [5]:
D_train, D_test, U_train, U_test = sklearn.model_selection.train_test_split(D, U,test_size=0.2)

In [6]:
def get_dataset():
    return D_train, D_test, U_train, U_test

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [8]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 52, 512)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "softplus", "relu"])
        layers += [out_features]
        activations.append(activation)
    layers += [output_size]
    params_MLP_trial = copy.deepcopy(params_MLP)
    params_MLP_trial['layer_dims'] = layers
    params_MLP_trial['activations'] = activations
    return MLP(params={'solver':None, 'method':params_MLP_trial})

In [9]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1, 101)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_test, U_train, U_test = get_dataset()

    # Training of the model.
    for epoch in range(1000):
        model.train()

        optimizer.zero_grad()
        output = model(D_train)
        loss = loss_fn(output, U_train)
        loss.backward()
        optimizer.step()

        loss_train = loss.item()
        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            output = model(D_test)
            loss = loss_fn(output, U_test)
            
        loss_val = loss.item()

        trial.report(loss_val, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return loss_val

In [10]:
study = optuna.create_study(direction="minimize")

[I 2023-07-31 15:18:41,397] A new study created in memory with name: no-name-b63376c3-b4f7-4000-b63f-8b8b6de6f79f


In [11]:
study.optimize(objective, n_trials=40, timeout=600)

[W 2023-07-31 15:18:49,748] Trial 0 failed with parameters: {'n_layers': 6, 'units_l0': 416, 'activation_l0': 'tanh', 'units_l1': 267, 'activation_l1': 'tanh', 'units_l2': 294, 'activation_l2': 'tanh', 'units_l3': 78, 'activation_l3': 'relu', 'units_l4': 296, 'activation_l4': 'relu', 'units_l5': 383, 'activation_l5': 'relu', 'optimizer': 'Adam', 'lr': 3.165316569795445e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/work/lechevaa/miniconda3/envs/CEMRACS_SciML/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_22603/3228726086.py", line 17, in objective
    output = model(D_train)
  File "/work/lechevaa/miniconda3/envs/CEMRACS_SciML/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/work/lechevaa/PycharmProjects/CEMRACS_SciML/methods/MLP.py", line 80, in forward
    x

KeyboardInterrupt: 

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [ ]:
print(activations, '\n', layer_dims, '\n', optimizer, '\n', lr)